In [24]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [25]:
dataset,info = tfds.load('tf_flowers',as_supervised=True,with_info=True)
dataset_size = info.splits['train'].num_examples #3670
class_names = info.features['label'].names #['dandelion','daisy']
n_class = info.features['label'].num_classes # 5

2025-03-06 12:04:11.750417: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 100%|██████████████████████████████████████████████████████████████| 5/5 [03:22<00:00, 40.52s/ file]

Dataset tf_flowers downloaded and prepared to /home/harry/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [28]:
test_set_raw,valid_set_raw,train_set_raw = tfds.load('tf_flowers',
                                                    split = ['train[:10%]','train[10%:25%]','train[25%:]'],
                                                    as_supervised = True)

In [38]:
batch_size = 32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224,width =224,crop_to_aspect_ratio = True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])

train_set = train_set_raw.map(lambda X,y:(preprocess(X),y))
train_set = train_set.shuffle(1000,seed=42).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)
test_set  = test_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)

In [39]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode = 'horizontal',seed = 42),
    tf.keras.layers.RandomRotation(factor=0.05,seed = 42),
    tf.keras.layers.RandomContrast(factor=0.2,seed = 42)
])

In [40]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet',include_top = False)
avg=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_class,activation='softmax')(avg)
model = tf.keras.Model(inputs=base_model.input,outputs=output)

In [41]:
for layer in base_model.layers:
    layer.trainable = False

In [42]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy',optimizer= optimizer,metrics=['accuracy'])
history = model.fit(train_set,validation_data = valid_set,epochs =3)

Epoch 1/3


2025-03-06 12:10:24.600670: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


86/86 ━━━━━━━━━━━━━━━━━━━━ 242s 3s/step - accuracy: 0.6975 - loss: 1.0837 - val_accuracy: 0.8258 - val_loss: 0.6878
Epoch 2/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 277s 3s/step - accuracy: 0.9149 - loss: 0.3437 - val_accuracy: 0.8693 - val_loss: 0.5803
Epoch 3/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 259s 3s/step - accuracy: 0.9299 - loss: 0.2237 - val_accuracy: 0.8693 - val_loss: 0.5896


###  Unfreezing some top layers and training again

In [43]:
for layer in base_model.layers[56:]:
    layer.trainable = True

In [45]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy',optimizer= optimizer,metrics=['accuracy'])
history = model.fit(train_set,validation_data = valid_set,epochs =3)

Epoch 1/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 550s 6s/step - accuracy: 0.8798 - loss: 0.3650 - val_accuracy: 0.8802 - val_loss: 0.6065
Epoch 2/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 462s 5s/step - accuracy: 0.9825 - loss: 0.0579 - val_accuracy: 0.9002 - val_loss: 0.3450
Epoch 3/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 504s 5s/step - accuracy: 0.9930 - loss: 0.0215 - val_accuracy: 0.9165 - val_loss: 0.2710


In [ ]:
##classification and localization in the image

In [49]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet',include_top = False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = tf.keras.layers.Dense(n_class,activation='softmax')(avg)
loc_output = tf.keras.layers.Dense(4)(avg)
model =  tf.keras.Model(inputs=base_model.input,outputs=[class_output,loc_output])

model.compile(loss=['sparse_categorical_crossentropy','mse'],
              loss_weights = [0.8,0.2],
              optimizer=optimizer,metrics=['accuracy'])